## Using Langchain to create a custom prompt

The _Retrieval_ portion or the _Retrieval Augmented Generation_ (RAG) pattern allows you to generate responses based on your data. 

Example:
    You're searching for information that can be compiled from multiple documents.

Instead of providing the links to all the documents alone and configuring highlighting for each piece.

That being said we have vector embeddings.


In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

from opensearchpy import OpenSearch

connection_string = os.getenv("OPENSEARCH_SERVICE_URI")

# Create the client with SSL/TLS enabled, but hostname verification disabled.
client = OpenSearch(connection_string, use_ssl=True, timeout=100)
client.info()

In [ ]:
from pprint import pprint

index_name = "openai_wikipedia_index"

# Create the client with SSL/TLS enabled, but hostname verification disabled.
client = OpenSearch(connection_string, use_ssl=True, timeout=100)

res = client.search(index=index_name, body={
    "_source": {},
    "query": {
        "match": {
            "text": {
                "query": "Pizza"
            }
        }
    }
})

print("\n".join([doc['_source']['title'] for doc in res["hits"]["hits"]]))

## Run semantic search queries with LangChain and OpenSearch

With the above embedding calculated, we can now run semantic searches against the OpenSearch index. We're using `knn` as query type and scan the content of the `content_vector` field.

After running the block below, we should see content semantically similar to the question. Expect documents based on Pineapples, Pizza, Hawaii, Italy, etc.

In [ ]:
import os
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

# Define model
EMBEDDING_MODEL = "text-embedding-ada-002"

# Define the Client
openaiclient = OpenAI(
    # This is the default and can be omitted
    api_key=os.getenv("OPENAI_API_KEY"),
)

# Define question
question = 'is ice cream a good option'

# Create embedding
question_embedding = openaiclient.embeddings.create(input=question + "pizza toppings", model=EMBEDDING_MODEL)

In [ ]:
opensearch_response = client.search(
  index = index_name,
  body = {
      "size": 5,
      "query" : {
        "knn" : {
          "content_vector":{
            "vector":  question_embedding.data[0].embedding,
            "k": 10
          }
       }
     }
  }
)

print("\n".join([doc['_source']['title'] for doc in opensearch_response['hits']['hits']]))


## Conclusion

OpenSearch is a powerful tool providing both text and vector search capabilities. Used alongside LangChain allows you to craft personalized AI applications able to augment the context based on semantic search. LangChain's extensive modularity allows you to choose your

You can try Aiven for OpenSearch, or any of the other Open Source tools, in the Aiven platform free trial by [signing up](https://go.aiven.io/openai-opensearch-signup).